In [29]:
import root_config as rc

rc.configure()

from detectdd.auth_bigquery import BigQueryClient
from detectdd.serializer import Serializer

try:
    cohort = Serializer().read_cohort()  # need to run 01-cohort.ipynb to produce the cohort
except FileNotFoundError:
    raise Exception("Need to run [01-cohort.ipynb] at least once to create the cohort file in the /out directory")


big_query = BigQueryClient.auth()

cohort.describe()


Loaded cohort from ..\out\cohort-full.out
<google.oauth2.credentials.Credentials object at 0x0000016513E8E560> lucky-curve-395616


,subject_id,hadm_id,stay_id,dose_b_time,event_count,num_icd_codes
count,13918.0,13918.0,13918.0,13918,13918.0,13918.0
mean,14962882.262107,24985829.341213,34988872.550582,2154-11-03 04:32:33.138382,2.140609,1.859103
min,10001884.0,20022095.0,30004144.0,2110-02-10 06:30:00,0.0,1.0
25%,12504186.0,22503715.0,32511600.0,2133-06-25 10:45:00,1.0,1.0
50%,14978206.0,25025631.0,34986873.0,2155-12-23 12:44:30,2.0,2.0
75%,17399295.0,27440064.0,37510396.25,2176-03-09 16:44:00,3.0,2.0
max,19995595.0,29997500.0,39987478.0,2208-06-15 19:48:00,9.0,10.0
std,2838772.908197,2855966.941545,2884324.657779,NaN,1.619336,1.087754


## kidney diagnoses

In [30]:
kidney_diagnoses_query = """
SELECT d.subject_id, d.hadm_id, d.icd_code, i.long_title
FROM `physionet-data.mimiciv_hosp.diagnoses_icd` d
JOIN `physionet-data.mimiciv_hosp.d_icd_diagnoses` i ON d.icd_code = i.icd_code
WHERE i.long_title LIKE '%kidney%'
"""

kidney_diagnoses_results = big_query.query(kidney_diagnoses_query).result().to_dataframe()


## kidney prescriptions

In [31]:
kidney_prescriptions_query = """
WITH kidney_patients AS (
    SELECT DISTINCT d.subject_id, d.hadm_id
    FROM `physionet-data.mimiciv_hosp.diagnoses_icd` d
    JOIN `physionet-data.mimiciv_hosp.d_icd_diagnoses` i ON d.icd_code = i.icd_code
    WHERE i.long_title LIKE '%kidney%'
    LIMIT 100000
)

SELECT p.subject_id, p.hadm_id, p.drug
FROM `physionet-data.mimiciv_hosp.prescriptions` p
JOIN kidney_patients kp ON p.subject_id = kp.subject_id AND p.hadm_id = kp.hadm_id
LIMIT 100000
"""

kidney_prescriptions_results = big_query.query(kidney_prescriptions_query).result().to_dataframe()


In [32]:
kidney_prescriptions_results

,subject_id,hadm_id,drug
0,10771068,24650455,Heparin
1,12723597,28612481,Heparin
2,15533039,28396110,Albuterol 0.083% Neb Soln 1Neb
3,10386303,23372208,Calcium Gluconate
4,11166600,25863147,EPINEPHrine (for dilution)
...,...,...,...
99995,18376342,23636482,Calcium Acetate
99996,16273932,25757574,Calcium Acetate
99997,17341130,28017025,Calcium Acetate
99998,18300417,26122760,Calcium Acetate
